## **BERT Embedding**

In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

#Our sentences we like to encode
sentences = ['Hello World!',
    'Go around the table',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = np.asarray(model.encode(sentences))

#Print the embeddings
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

print(embeddings.shape)

(3, 384)


In [2]:
from sklearn.metrics.pairwise import cosine_similarity

query = "table"

query_embeddings = np.asarray(model.encode(query)).reshape(1, -1)


print(query_embeddings.shape)

compare_similarity = cosine_similarity(embeddings, query_embeddings)

top_n_idx = np.argmax(compare_similarity)

# Check if similarity score is above a threshold (e.g., 0.6)
if compare_similarity[top_n_idx] > 0.6:
    most_similar_sentence = sentences[top_n_idx]
    similarity_score = compare_similarity[top_n_idx]

    # Display the most similar sentence and its similarity score
    print("Most similar sentence:", most_similar_sentence)
    print("Similarity score:", similarity_score)
else:
    print("No similar sentence found.")

(1, 384)
Most similar sentence: Go around the table
Similarity score: [0.6397144]


- Handles the object
- Handles static
- Extend with dynamic

In [3]:
import json

with open("../data/mapper.json", "r") as f:
    mapper = json.load(f)['intents']

dynamic_map, static_map = mapper[:2], mapper[2:]

with open("../data/source.json", 'r') as f:
    data = json.load(f)


dynamic_patterns = set()
dynamic_map_patterns = [(mapper_feature, pattern) 
                        for mapper_feature in dynamic_map
                        for pattern in mapper_feature['patterns']]
for index in data:
    object_list = data[index]['object_list']
    for obj in object_list:
        for mapper_feature, pattern in dynamic_map_patterns:
            dynamic_patterns.add((pattern.replace('{obj}', obj), mapper_feature['feature'].replace('{obj}', obj)))

dynamic_patterns = list(dynamic_patterns) # (pattern, feature)

static_patterns = []
for mapper_feature in static_map:
    for pattern in mapper_feature['patterns']:
        static_patterns.append((pattern, mapper_feature['feature']))

mapper_patterns =  dynamic_patterns + static_patterns

print(f"mapper_patterns size: {len(mapper_patterns)}")
print(f"mapper_patterns: {mapper_patterns}")


mapper_patterns size: 347
mapper_patterns: [('Decrease distance to pen', 'pen distance decrease'), ('Keep a bigger distance from notebook', 'notebook distance increase'), ('Move further away from glass', 'glass distance increase'), ('Keep a bigger distance from can', 'can distance increase'), ('Avoid monitor', 'monitor distance increase'), ('Stay away from book', 'book distance increase'), ('Move closer to textbook', 'textbook distance decrease'), ('Keep a smaller distance from spoon', 'spoon distance decrease'), ('Avoid box', 'box distance increase'), ('Increase distance to microwave', 'microwave distance increase'), ('Decrease distance to vase', 'vase distance decrease'), ('Keep a bigger distance from book', 'book distance increase'), ('Stay close to book', 'book distance decrease'), ('Move further away from orange', 'orange distance increase'), ('Keep a bigger distance from jar', 'jar distance increase'), ('Stay away from can', 'can distance increase'), ('Move further away from micr

In [4]:
mapper_patterns[:20]

[('Decrease distance to pen', 'pen distance decrease'),
 ('Keep a bigger distance from notebook', 'notebook distance increase'),
 ('Move further away from glass', 'glass distance increase'),
 ('Keep a bigger distance from can', 'can distance increase'),
 ('Avoid monitor', 'monitor distance increase'),
 ('Stay away from book', 'book distance increase'),
 ('Move closer to textbook', 'textbook distance decrease'),
 ('Keep a smaller distance from spoon', 'spoon distance decrease'),
 ('Avoid box', 'box distance increase'),
 ('Increase distance to microwave', 'microwave distance increase'),
 ('Decrease distance to vase', 'vase distance decrease'),
 ('Keep a bigger distance from book', 'book distance increase'),
 ('Stay close to book', 'book distance decrease'),
 ('Move further away from orange', 'orange distance increase'),
 ('Keep a bigger distance from jar', 'jar distance increase'),
 ('Stay away from can', 'can distance increase'),
 ('Move further away from microwave', 'microwave distance

### USING BERT ###

In [5]:
token_count = len(model.tokenizer.encode("Environmentalist", add_special_tokens=True))
token_count

4

In [6]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

sentences = [pattern[0] for pattern in mapper_patterns]
pattern_embeddings = np.asarray(model.encode(sentences))

token_ids = [model.tokenizer.encode(pattern, add_special_tokens=True) for pattern in mapper_patterns]

mapper_tokens = sum(len(tokens) for tokens in token_ids)


# query = "Take a step forward"

result = {}
total_tokens = 0 + mapper_tokens
for idx, each in enumerate(data.values()):
    temp = {}
    query = each['correction'] 
    print(f"QUERY: {query}")
    query_embeddings = np.asarray(model.encode(query)).reshape(1, -1)
    token = len(model.tokenizer.encode(query, add_special_tokens=True))
    total_tokens += token

    compare_similarity = cosine_similarity(pattern_embeddings, query_embeddings)

    top_n_idx = np.argmax(compare_similarity)

    # Check if similarity score is above a threshold (e.g., 0.6)
    if compare_similarity[top_n_idx] > 0.6:
        most_similar_sentence, predicted_feature = mapper_patterns[top_n_idx]
        confidence_score = round(float(compare_similarity[top_n_idx]), 2)

        print(f"PREDICTED FEATURE: {predicted_feature} (confidence: {confidence_score})")
        print(f"CLOSEST SENTENCE: {most_similar_sentence}")
        
    else:
        print("No similar sentence found.")
    
    temp['correction'] = query
    temp['tokens'] = token
    temp['closest_sentence'] = most_similar_sentence
    temp['confidence_score'] = confidence_score
    temp['predicted_feature'] = predicted_feature
    temp['gt_feature'] = each['features']
    result[idx + 1] = temp

    print(f"TOKENS: {token}")
    print("="*150)
    print()

print(f"total_tokens: {total_tokens}")

with open("../result/sensim/bert.json", "w") as f:
    json.dump(result, f, indent=4)

QUERY: Get close to bottle and turn right then move closer to apple and stay away from book
PREDICTED FEATURE: bottle distance increase (confidence: 0.61)
CLOSEST SENTENCE: Move further away from bottle
TOKENS: 19

QUERY: Approach the vase and move left then get nearer to the orange and stay down
PREDICTED FEATURE: vase distance decrease (confidence: 0.72)
CLOSEST SENTENCE: Move closer to vase
TOKENS: 17

QUERY: Reduce the distance to the box and turn left then get a bit closer to the banana but avoid the pen


/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)


PREDICTED FEATURE: banana distance decrease (confidence: 0.71)
CLOSEST SENTENCE: Move closer to banana
TOKENS: 23

QUERY: Move closer to the cup and go right then keep close to the pear but step back from the notebook
PREDICTED FEATURE: pear distance decrease (confidence: 0.68)
CLOSEST SENTENCE: Move closer to pear
TOKENS: 22

QUERY: Stay close to the jar and move leftward then decrease the distance to the watermelon but avoid the magazine
PREDICTED FEATURE: watermelon distance increase (confidence: 0.73)
CLOSEST SENTENCE: Keep a bigger distance from watermelon
TOKENS: 24

QUERY: Get nearer to the bottle and approach the monitor then get a little bit lower and stay down
PREDICTED FEATURE: bottle distance decrease (confidence: 0.68)
CLOSEST SENTENCE: Move closer to bottle
TOKENS: 20

QUERY: Retreat from the table and move rightwards then move down and keep your distance from the textbook


/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_2

PREDICTED FEATURE: textbook distance increase (confidence: 0.71)
CLOSEST SENTENCE: Move further away from textbook
TOKENS: 21

QUERY: Get closer to the plate and go forward then move closer to the peach but steer clear of the journal
PREDICTED FEATURE: peach distance decrease (confidence: 0.68)
CLOSEST SENTENCE: Move closer to peach
TOKENS: 22

QUERY: Retreat from the glass and stay left then increase the distance from the mango and move a bit further away from the diary
PREDICTED FEATURE: mango distance increase (confidence: 0.65)
CLOSEST SENTENCE: Move further away from mango
TOKENS: 25

QUERY: Keep a greater distance from the bowl and move right then move up and avoid the notebook
PREDICTED FEATURE: bowl distance increase (confidence: 0.67)
CLOSEST SENTENCE: Move further away from bowl
TOKENS: 19

QUERY: Traverse right and get closer to the desktop then stay elevated and go backwards
PREDICTED FEATURE: desktop distance increase (confidence: 0.74)
CLOSEST SENTENCE: Move further away 

...

/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_2

PREDICTED FEATURE: Y-cartesian decrease (confidence: 0.79)
CLOSEST SENTENCE: Move backward
TOKENS: 8

QUERY: Move backward and then move up
PREDICTED FEATURE: Y-cartesian decrease (confidence: 0.86)
CLOSEST SENTENCE: Move backward
TOKENS: 8

QUERY: Approach the cup but avoid the scissors and keep left
PREDICTED FEATURE: scissors distance increase (confidence: 0.68)
CLOSEST SENTENCE: Avoid scissors
TOKENS: 12

QUERY: Go close to the banana but keep a distance from apple
PREDICTED FEATURE: banana distance decrease (confidence: 0.87)
CLOSEST SENTENCE: Stay close to banana
TOKENS: 13

QUERY: Move towards the cup and keep left
PREDICTED FEATURE: cup distance decrease (confidence: 0.8)
CLOSEST SENTENCE: Move closer to cup
TOKENS: 9

QUERY: Move a little bit forward


/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)
/tmp/ipykernel_2

PREDICTED FEATURE: Y-cartesian increase (confidence: 0.8)
CLOSEST SENTENCE: Move forward
TOKENS: 7

QUERY: Stay nearer to cup but keep a distance from the scissors
PREDICTED FEATURE: scissors distance decrease (confidence: 0.81)
CLOSEST SENTENCE: Stay close to scissors
TOKENS: 13

QUERY: Stay far from the book and keep right while moving forward
PREDICTED FEATURE: book distance increase (confidence: 0.81)
CLOSEST SENTENCE: Move further away from book
TOKENS: 13

QUERY: Move towards the book and keep back from the banana
PREDICTED FEATURE: banana distance increase (confidence: 0.71)
CLOSEST SENTENCE: Move further away from banana
TOKENS: 12

QUERY: Take a step forward
PREDICTED FEATURE: Y-cartesian increase (confidence: 0.82)
CLOSEST SENTENCE: Move forward
TOKENS: 6

total_tokens: 4689


/tmp/ipykernel_24141/3172109677.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  confidence_score = round(float(compare_similarity[top_n_idx]), 2)


In [19]:
import pandas as pd 

df = pd.read_json("../result/sensim/bert.json").T
df.head()

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature
1,Get close to bottle and turn right then move c...,19,Move further away from bottle,0.61,bottle distance increase,"[bottle distance decrease, X-cartesian increas..."
2,Approach the vase and move left then get neare...,17,Move closer to vase,0.72,vase distance decrease,"[vase distance decrease, X-cartesian decrease,..."
3,Reduce the distance to the box and turn left t...,23,Move closer to banana,0.71,banana distance decrease,"[box distance decrease, X-cartesian decrease, ..."
4,Move closer to the cup and go right then keep ...,22,Move closer to pear,0.68,pear distance decrease,"[cup distance decrease, X-cartesian increase, ..."
5,Stay close to the jar and move leftward then d...,24,Keep a bigger distance from watermelon,0.73,watermelon distance increase,"[jar distance decrease, X-cartesian decrease, ..."


In [20]:
def is_prediction_correct(row):
    if len(row['gt_feature']) == 1:
        return row['predicted_feature'].lower() in [x.lower() for x in row['gt_feature']]
    return False
    

# Apply the function to the DataFrame
df['matches'] = df.apply(is_prediction_correct, axis=1)

df.head(1)

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature,matches
1,Get close to bottle and turn right then move c...,19,Move further away from bottle,0.61,bottle distance increase,"[bottle distance decrease, X-cartesian increas...",False


In [9]:
df.to_csv("../result/sensim/bert.csv", index=True)

In [10]:
df = pd.read_csv("../result/sensim/bert.csv", index_col=0)
df.tail()

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature,matches
96,Move a little bit forward,7,Move forward,0.80,Y-cartesian increase,['Y-cartesian increase'],True
97,Stay nearer to cup but keep a distance from th...,13,Stay close to scissors,0.81,scissors distance decrease,"['Cup distance decrease', 'Scissors distance i...",False
98,Stay far from the book and keep right while mo...,13,Move further away from book,0.81,book distance increase,"['Book distance increase', 'X-cartesian increa...",False
99,Move towards the book and keep back from the b...,12,Move further away from banana,0.71,banana distance increase,"['book distance decrease', 'banana distance in...",False
100,Take a step forward,6,Move forward,0.82,Y-cartesian increase,['Y-cartesian increase'],True


In [22]:
import ast 

# df['gt_feature'] = df['gt_feature'].apply(ast.literal_eval)
single_cmd = df[df['gt_feature'].apply(lambda x: len(x) == 1)]
chain_cmd = df[df['gt_feature'].apply(lambda x: len(x) > 1)]

print(f"single_cmd: {len(single_cmd)}")
print(f"chain_cmd: {len(chain_cmd)}")

single_cmd: 55
chain_cmd: 45


In [23]:
single_cmd_indices = single_cmd.index.tolist()
print(len(single_cmd_indices))

55


In [24]:
df['cmd_type'] = 'complex' 
df.loc[single_cmd_indices, 'cmd_type'] = 'single'
df.head()

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature,matches,cmd_type
1,Get close to bottle and turn right then move c...,19,Move further away from bottle,0.61,bottle distance increase,"[bottle distance decrease, X-cartesian increas...",False,complex
2,Approach the vase and move left then get neare...,17,Move closer to vase,0.72,vase distance decrease,"[vase distance decrease, X-cartesian decrease,...",False,complex
3,Reduce the distance to the box and turn left t...,23,Move closer to banana,0.71,banana distance decrease,"[box distance decrease, X-cartesian decrease, ...",False,complex
4,Move closer to the cup and go right then keep ...,22,Move closer to pear,0.68,pear distance decrease,"[cup distance decrease, X-cartesian increase, ...",False,complex
5,Stay close to the jar and move leftward then d...,24,Keep a bigger distance from watermelon,0.73,watermelon distance increase,"[jar distance decrease, X-cartesian decrease, ...",False,complex


In [25]:
value_counts_dict = {
    'single_cmd': single_cmd['matches'].value_counts(),
    'chain_cmd': chain_cmd['matches'].value_counts()
}
df_stat = pd.DataFrame(value_counts_dict)
df_stat = df_stat.fillna(0)

df_stat

,single_cmd,chain_cmd
matches,,
False,7,45.0
True,48,0.0


In [13]:
markdown_table = df.to_markdown(buf=None, mode='pipe', index=True)
res_table = df_stat.to_markdown(buf=None, mode='pipe', index=True)
with open('../result/sensim/bert.md', 'w') as file:
    file.write("## Result - BERT (no prompts)\n\n")
    file.write(markdown_table)
    file.write("\n\n")
    file.write("## Stats\n\n")
    file.write(res_table)

### USING ADA ###

In [14]:
import json

with open("../data/mapper.json", "r") as f:
    mapper = json.load(f)['intents']

dynamic_map, static_map = mapper[:2], mapper[2:]

with open("../data/source.json", 'r') as f:
    data = json.load(f)


dynamic_patterns = set()
dynamic_map_patterns = [(mapper_feature, pattern) 
                        for mapper_feature in dynamic_map
                        for pattern in mapper_feature['patterns']]
for index in data:
    object_list = data[index]['object_list']
    for obj in object_list:
        for mapper_feature, pattern in dynamic_map_patterns:
            dynamic_patterns.add((pattern.replace('{obj}', obj), mapper_feature['feature'].replace('{obj}', obj)))

dynamic_patterns = list(dynamic_patterns) # (pattern, feature)

static_patterns = []
for mapper_feature in static_map:
    for pattern in mapper_feature['patterns']:
        static_patterns.append((pattern, mapper_feature['feature']))

mapper_patterns =  dynamic_patterns + static_patterns

print(f"mapper_patterns size: {len(mapper_patterns)}")
print(f"mapper_patterns: {mapper_patterns}")

mapper_patterns size: 347
mapper_patterns: [('Decrease distance to pen', 'pen distance decrease'), ('Keep a bigger distance from notebook', 'notebook distance increase'), ('Move further away from glass', 'glass distance increase'), ('Keep a bigger distance from can', 'can distance increase'), ('Avoid monitor', 'monitor distance increase'), ('Stay away from book', 'book distance increase'), ('Move closer to textbook', 'textbook distance decrease'), ('Keep a smaller distance from spoon', 'spoon distance decrease'), ('Avoid box', 'box distance increase'), ('Increase distance to microwave', 'microwave distance increase'), ('Decrease distance to vase', 'vase distance decrease'), ('Keep a bigger distance from book', 'book distance increase'), ('Stay close to book', 'book distance decrease'), ('Move further away from orange', 'orange distance increase'), ('Keep a bigger distance from jar', 'jar distance increase'), ('Stay away from can', 'can distance increase'), ('Move further away from micr

In [6]:
import openai
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

def get_embeddings(sentences):
    response = openai.embeddings.create(
        input=sentences,
        model="text-embedding-ada-002"
    )

    token_count = response.usage.total_tokens
    embeddings = np.array([item.embedding for item in response.data])
   
    return np.asarray(embeddings), token_count

source_sentence = "Stay away from the bottle"
test_sentences = [
    "Go close to the bottle",
    "Stay closer to the bottle",
]

source_embeddings, _ = get_embeddings(source_sentence) 

print(f"Source sentence: {source_sentence}")
for sentence in test_sentences:
    test_embeddings, _ = get_embeddings(sentence)
    similarity_score = cosine_similarity(source_embeddings, test_embeddings)
    print(f"Sentence to compare: {sentence} -> {similarity_score[0][0]}")


Source sentence: Stay away from the bottle
Sentence to compare: Go close to the bottle -> 0.921524056178013
Sentence to compare: Stay closer to the bottle -> 0.938044284076677


In [27]:
import openai
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to get embeddings using OpenAI's API
def get_embeddings(sentences):
    response = openai.embeddings.create(
        input=sentences,
        model="text-embedding-ada-002"
    )


    token_count = response.usage.total_tokens
    # if isinstance(sentences, str) or len(sentences) == 1:
    #     # Reshape in case of single sentence (1D array)
    #     embeddings = np.array(response.data[0].embedding).reshape(1, -1)
    #     print(embeddings.shape)
    # else:
        # Convert list of embedding lists to 2D array
    embeddings = np.array([item.embedding for item in response.data])
   
    return np.asarray(embeddings), token_count

# get_embeddings("Hello World")
sentences = [pattern[0] for pattern in mapper_patterns]

embeddings_file_path = "../data/ada_pattern_embeddings.npy"
try:
    pattern_embeddings = np.load(embeddings_file_path)
    print("Loading from existing ada_pattern_embeddings.npy file...")
except FileNotFoundError:
    print("No existing ada_pattern_embeddings.npy file found.\nFetching and saving embeddings...")
    pattern_embeddings, mapper_tokens = get_embeddings(sentences)  # Replace this with actual API call
    np.save(embeddings_file_path, pattern_embeddings)


result = {}
total_tokens = 0 + mapper_tokens
for idx, each in enumerate(data.values()):
    temp = {}
    query = each['correction'] 
    print(f"QUERY: {query}")
    query_embeddings, query_tokens = get_embeddings(query) 
    print(f"QUERY EMBEDDING SIZE: {query_embeddings.shape}")

    print(f"QUERY: {query}")
    print(f"TOKENS: {query_tokens}")

    compare_similarity = cosine_similarity(pattern_embeddings, query_embeddings)

    top_n_idx = np.argmax(compare_similarity)

    if compare_similarity[top_n_idx] > 0.6:
        most_similar_sentence, predicted_feature = mapper_patterns[top_n_idx]
        similarity_score = compare_similarity[top_n_idx][0] 
        confidence_score = round(float(similarity_score), 2)

        print(f"PREDICTED FEATURE: {predicted_feature} (confidence: {confidence_score})")
        print(f"CLOSEST SENTENCE: {most_similar_sentence}")
    else:
        print("No similar sentence found.")

    temp['correction'] = query
    temp['tokens'] = query_tokens
    temp['closest_sentence'] = most_similar_sentence
    temp['confidence_score'] = confidence_score
    temp['predicted_feature'] = predicted_feature
    temp['gt_feature'] = each['features']
    result[idx + 1] = temp

    total_tokens += query_tokens

    print(f"TOKENS: {query_tokens}")
    print("="*150)
    print()


print(f"total_tokens: {total_tokens}")

with open("../result/sensim/ada.json", "w") as f:
    json.dump(result, f, indent=4)


No existing ada_pattern_embeddings.npy file found.
Fetching and saving embeddings...
QUERY: Get close to bottle and turn right then move closer to apple and stay away from book
QUERY EMBEDDING SIZE: (1, 1536)
QUERY: Get close to bottle and turn right then move closer to apple and stay away from book
TOKENS: 17
PREDICTED FEATURE: bottle distance decrease (confidence: 0.9)
CLOSEST SENTENCE: Move closer to bottle
TOKENS: 17

QUERY: Approach the vase and move left then get nearer to the orange and stay down
QUERY EMBEDDING SIZE: (1, 1536)
QUERY: Approach the vase and move left then get nearer to the orange and stay down
TOKENS: 16
PREDICTED FEATURE: vase distance decrease (confidence: 0.89)
CLOSEST SENTENCE: Move closer to vase
TOKENS: 16

QUERY: Reduce the distance to the box and turn left then get a bit closer to the banana but avoid the pen
QUERY EMBEDDING SIZE: (1, 1536)
QUERY: Reduce the distance to the box and turn left then get a bit closer to the banana but avoid the pen
TOKENS: 21

In [8]:
import pandas as pd 

def is_prediction_correct(row):
    if len(row['gt_feature']) == 1:
        return row['predicted_feature'].lower() in [x.lower() for x in row['gt_feature']]
    return False

df = pd.read_json("../result/sensim/ada.json").T

df['matches'] = df.apply(is_prediction_correct, axis=1)

df.head(1)

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature,matches
1,Get close to bottle and turn right then move c...,17,Move closer to bottle,0.9,bottle distance decrease,"[bottle distance decrease, X-cartesian increas...",False


In [9]:
import ast 

# df['gt_feature'] = df['gt_feature'].apply(ast.literal_eval)
single_cmd = df[df['gt_feature'].apply(lambda x: len(x) == 1)]
chain_cmd = df[df['gt_feature'].apply(lambda x: len(x) > 1)]

print(f"single_cmd: {len(single_cmd)}")
print(f"chain_cmd: {len(chain_cmd)}")

single_cmd: 55
chain_cmd: 45


In [10]:
single_cmd.head()

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature,matches
29,move up,2,Move up,0.98,Z-cartesian increase,[Z-cartesian increase],True
30,move right,2,Move right,0.98,X-cartesian increase,[X-cartesian increase],True
31,keep close to pineapple,4,Stay close to pineapple,0.98,pineapple distance decrease,[Pineapple distance decrease],True
32,move near the bottle,4,Move closer to bottle,0.97,bottle distance decrease,[Bottle distance decrease],True
33,move near the pineapple,4,Move closer to pineapple,0.97,pineapple distance decrease,[Pineapple distance decrease],True


In [11]:
single_cmd_indices = single_cmd.index.tolist()
print(len(single_cmd_indices))

55


In [12]:
print(single_cmd_indices)

[29, 30, 31, 32, 33, 34, 35, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85, 87, 88, 89, 90, 96, 100]


In [13]:
df['cmd_type'] = 'complex' 
df.loc[single_cmd_indices, 'cmd_type'] = 'single'
df.head()

,correction,tokens,closest_sentence,confidence_score,predicted_feature,gt_feature,matches,cmd_type
1,Get close to bottle and turn right then move c...,17,Move closer to bottle,0.9,bottle distance decrease,"[bottle distance decrease, X-cartesian increas...",False,complex
2,Approach the vase and move left then get neare...,16,Move closer to vase,0.89,vase distance decrease,"[vase distance decrease, X-cartesian decrease,...",False,complex
3,Reduce the distance to the box and turn left t...,21,Decrease distance to banana,0.91,banana distance decrease,"[box distance decrease, X-cartesian decrease, ...",False,complex
4,Move closer to the cup and go right then keep ...,20,Move closer to notebook,0.9,notebook distance decrease,"[cup distance decrease, X-cartesian increase, ...",False,complex
5,Stay close to the jar and move leftward then d...,21,Decrease distance to watermelon,0.9,watermelon distance decrease,"[jar distance decrease, X-cartesian decrease, ...",False,complex


In [14]:
single_cmd['matches'].value_counts()

matches
True    55
Name: count, dtype: int64

In [15]:
chain_cmd['matches'].value_counts()

matches
False    45
Name: count, dtype: int64

In [16]:
value_counts_dict = {
    'single_cmd': single_cmd['matches'].value_counts(),
    'chain_cmd': chain_cmd['matches'].value_counts()
}
df_stat = pd.DataFrame(value_counts_dict)
df_stat = df_stat.fillna(0)

df_stat

,single_cmd,chain_cmd
matches,,
False,0.0,45.0
True,55.0,0.0


In [17]:
markdown_table = df.to_markdown(buf=None, mode='pipe', index=True)
res_table = df_stat.to_markdown(buf=None, mode='pipe', index=True)
with open('../result/sensim/ada.md', 'w') as file:
    file.write(markdown_table)
    file.write("\n\n")
    file.write(res_table)

In [18]:
# import requests
# import json
# import os
# import numpy as np
# from dotenv import load_dotenv

# load_dotenv()

# api_key = os.getenv("OPENAI_API_KEY")
# headers = {
#     'Authorization': f'Bearer {api_key}',
#     'Content-Type': 'application/json',
# }

# data = {
#     "input": "Move forward",
#     "model": "text-embedding-ada-002",
#     "encoding_format": "float",
# }

# response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, data=json.dumps(data))

# if response.status_code == 200:
#     print("Embedding request successful.")
#     result = response.json()
#     print(np.asarray(result['data'][0]['embedding']).shape)
# else:
#     print(f"Failed to retrieve embeddings. Status code: {response.status_code}")
#     print(response.text)
